In [1]:
import numpy as np
import pandas as pd

In [2]:
with open("twitter-datasets/twitter-datasets/train_neg.txt") as f:
    neg_txt = [line for line in f]
    neg_df = pd.DataFrame(neg_txt, columns=['line'])
    print(neg_df.head())
f.close()

                                                line
0  vinco tresorpack 6 ( difficulty 10 of 10 objec...
1  glad i dot have taks tomorrow ! ! #thankful #s...
2  1-3 vs celtics in the regular season = were fu...
3  <user> i could actually kill that girl i'm so ...
4  <user> <user> <user> i find that very hard to ...


In [3]:
with open("twitter-datasets/twitter-datasets/train_pos.txt") as f:
    pos_txt = [line for line in f]
    pos_df = pd.DataFrame(pos_txt, columns=['line'])
    print(pos_df.head())
f.close

                                                line
0  <user> i dunno justin read my mention or not ....
1  because your logic is so dumb , i won't even c...
2  " <user> just put casper in a box ! " looved t...
3  <user> <user> thanks sir > > don't trip lil ma...
4  visiting my brother tmr is the bestest birthda...


<function TextIOWrapper.close()>

In [4]:
pos_df['label'] = 1
neg_df['label'] = 0

In [5]:
full_df = pd.concat([pos_df,neg_df],ignore_index=True)

In [6]:
import re
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Samuel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from tqdm import tqdm
corpus = []
for i in tqdm(range(len(full_df))):
  review = re.sub('[^a-zA-Z]', ' ', full_df['line'][i]) 
  review = review.lower() 
  review = review.split() 
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
corpus

100%|██████████| 200000/200000 [02:15<00:00, 1475.46it/s]


['user dunno justin read mention not justin god know hope follow believ',
 'logic dumb even crop name photo tsk url',
 'user put casper box loov battl crakkbitch',
 'user user thank sir trip lil mama keep doin ya thang',
 'visit brother tmr bestest birthday gift eveerrr',
 'user yay lifecomplet tweet facebook let know pleas',
 'user dnextalbumtitl feel rollercoast life song cocept life yolo becom famou followmeplz x',
 'workin hard hardli workin rt user harde futur cowork user',
 'user saw repli bit',
 'belong',
 'user anddd cheer nation',
 'send invit shop line find everyth need without leav home url',
 'woke finna go church',
 'user agre day left tho',
 'monet katemelo',
 'like dammm user lexi u got lot say ur twitter lol',
 'grate today dream fulfil heart full first complet track arriv back new york yeslord',
 'user home affair shall later',
 'barca bout beat real madrid saturday doe',
 'user lot part asia especi rat live countri live grain suppos quit tasti',
 'user user even sleep

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features= 1500)
X = tv.fit_transform(corpus).toarray()
y = full_df.iloc[:,-1].values

In [24]:
!pip install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = full_df.iloc[:,-1].values

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 647.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.0 MB 3.2 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.0 MB 5.7 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/11.0 MB 6.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/11.0 MB 9.6 MB/s eta 0:00:02
   ---------- ----------------------------- 2.8/11.0 MB 16.2 MB/s eta 0:00:01
   --------------------- ------------------ 5.8/11.0 MB 28.5 MB/s eta 0:00:01
   ------------------------------------ --- 9.9/11.0 MB 42.3 MB/s eta 0:00:01
   --------

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
classifiers = [GaussianNB(),LogisticRegressionCV(),RandomForestClassifier(),KNeighborsClassifier()]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    print(accuracy_score(classifier.predict(X_test),y_test))

0.6843
0.76935
0.7884
0.69495


In [33]:
accuracy_score(classifier.predict(X_test),y_test)

0.66645

In [9]:
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model_embd = SentenceTransformer("bert-base-nli-mean-tokens").to('cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [8]:
from tqdm import tqdm
embedings = []
for i in tqdm(range(len(full_df))):
    embedings.append(model_embd.encode(full_df['line'][i]))

  1%|          | 1210/200000 [00:45<2:04:31, 26.61it/s]


KeyboardInterrupt: 

In [23]:
import numpy as np